In [ ]:
pip install pandas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# List all contents of the mounted drive
!ls /content/drive/


MyDrive  Shareddrives


In [ ]:
# List contents of the "Shared drives" folder
!ls /content/drive/Shareddrives/

# Navigate to a specific shared drive
shared_drive_name = "hackathon"
!ls /content/drive/Shareddrives/{shared_drive_name}/data

# /content/drive/Shareddrives/hackathon/data/Data_DeportistasBeneficiadosPAD_0_0.csv


'comunicaciones '   hackathon
 Ambulatorio				   Enaho2023-100-vivienda.csv
 CATALOGO_CIE10.csv			   Enaho2023-604-transportesycomunicaciones.csv
 ConsultaB2_2022_II_v1.csv		   ubigeos.csv
 Data_DeportistasBeneficiadosPAD_0_0.csv  'url Mayumi.gdoc'
'Data Limpia Josue'


In [ ]:
import pandas as pd

data = '/content/drive/Shareddrives/hackathon/data/Enaho2023-100-vivienda.csv'

# Leer el archivo CSV
df = pd.read_csv(data, encoding="ISO-8859-1", na_values=['', ' ', 'NaN', 'NULL', 'N/A'],
                 keep_default_na=True,
                 skipinitialspace=True )

# Resumen de los datos (incluye el conteo de valores no nulos)
print(df.info())

valores_nulos_por_columna = df.isnull().sum()

# Filtra las columnas con más de 1 valor nulo
columnas_con_mas_de_1_nulo = valores_nulos_por_columna[valores_nulos_por_columna > 1]

# Imprime las columnas y la cantidad de valores nulos
print(columnas_con_mas_de_1_nulo)


In [ ]:
columna = 'P101'

nulos_en_columna = df[columna].isnull().sum()

print(f"La columna '{columna}' tiene {nulos_en_columna} valores nulos.")

La columna 'P101' tiene 10865 valores nulos.


In [ ]:
# Separacion de datos:

# Columnas de interés
columnas_interes = ['UBIGEO', 'P101', 'P105A', 'P106A', 'P110', 'P1144']

# Rellenar valores nulos (ejemplo usando la mediana para columnas numéricas y 'Unknown' para categórica)
df['UBIGEO'].fillna('Unknown', inplace=True)
for column in ['P101', 'P105A', 'P106A', 'P110', 'P1144']:
    df[column].fillna(df[column].median(), inplace=True)

# Crear un nuevo DataFrame con las columnas de interés
df_interes = df[columnas_interes]

# Verificar el nuevo DataFrame
print(df_interes.head())
print(df_interes.isnull().sum())

# Guardar el nuevo DataFrame en un archivo CSV
# df_interes.to_csv('nuevo_df_interes.csv', index=False)
df_interes

   UBIGEO  P101  P105A  P106A  P110  P1144
0   10201   4.0    1.0    2.0   2.0    1.0
1   10201   1.0    2.0    1.0   1.0    1.0
2   10201   1.0    1.0    2.0   1.0    1.0
3   10201   4.0    2.0    1.0   2.0    1.0
4   10201   4.0    2.0    1.0   2.0    1.0
UBIGEO    0
P101      0
P105A     0
P106A     0
P110      0
P1144     0
dtype: int64


,UBIGEO,P101,P105A,P106A,P110,P1144
0,10201,4.0,1.0,2.0,2.0,1.0
1,10201,1.0,2.0,1.0,1.0,1.0
2,10201,1.0,1.0,2.0,1.0,1.0
3,10201,4.0,2.0,1.0,2.0,1.0
4,10201,4.0,2.0,1.0,2.0,1.0
...,...,...,...,...,...,...
44373,10503,1.0,2.0,2.0,1.0,1.0
44374,10503,1.0,2.0,2.0,1.0,1.0
44375,10503,1.0,2.0,2.0,1.0,1.0
44376,10503,1.0,1.0,2.0,1.0,1.0


In [ ]:
from google.colab import files
dataubigeos = '/content/drive/Shareddrives/hackathon/data/ubigeos.csv'

# Cargar el segundo CSV que contiene información de departamentos
df_ubigeos = pd.read_csv(dataubigeos, encoding="ISO-8859-1")

# Realizar el merge usando la columna UBIGEO en df_interes y IDDIST en df_ubigeos
df_final = pd.merge(df_interes, df_ubigeos[['IDDIST', 'NOMBDEP']], left_on='UBIGEO', right_on='IDDIST', how='left')

# Eliminar la columna IDDIST si no es necesaria
df_final.drop(columns=['IDDIST'], inplace=True)

# Verificar el DataFrame final
print(df_final.head())

# Guardar el nuevo DataFrame en un archivo CSV
df_final.to_csv('ENAHO-100-condeptos.csv', index=False)

files.download('ENAHO-100-condeptos.csv')

   UBIGEO  P101  P105A  P106A  P110  P1144   NOMBDEP
0   10201   4.0    1.0    2.0   2.0    1.0  AMAZONAS
1   10201   1.0    2.0    1.0   1.0    1.0  AMAZONAS
2   10201   1.0    1.0    2.0   1.0    1.0  AMAZONAS
3   10201   4.0    2.0    1.0   2.0    1.0  AMAZONAS
4   10201   4.0    2.0    1.0   2.0    1.0  AMAZONAS


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Añadir la columna AÑO
df_final['ANHO'] = 2023

# Renombrar columnas
df_final.rename(columns={
    'NOMBDEP': 'DEPARTAMENTO',
    'P101': 'TIPOVIVIENDA',
    'P105A': 'PERTENENCIAVIVIENDA',
    'P106A': 'TIENETITULO',
    'P110': 'PROVENIENCIADEAGUA',
    'P1144': 'INTERNET'
}, inplace=True)

# Mapeo de valores
tipovivienda_map = {
    1.0: 'Casa independiente',
    2.0: 'Departamento en edificio',
    3.0: 'Vivienda en quinta',
    4.0: 'Vivienda en casa de vecindad (Callejon, solar o corralon)',
    5.0: 'Choza o cabanha',
    6.0: 'Vivienda improvisada',
    7.0: 'Local no destinado para habitacion humana',
    8.0: 'Otro'
}
df_final['TIPOVIVIENDA'] = df_final['TIPOVIVIENDA'].map(tipovivienda_map)

pertenenciavivienda_map = {
    1.0: 'Alquilada',
    2.0: 'Propia, totalmente pagada',
    3.0: 'Propia, por invasión',
    4.0: 'Propia, comprandola a plazos',
    5.0: 'Cedida por el centro de trabajo',
    6.0: 'Cedida por otro hogar o institucion',
    7.0: 'Otra forma',
    9.0: 'Missing Value'
}
df_final['PERTENENCIAVIVIENDA'] = df_final['PERTENENCIAVIVIENDA'].map(pertenenciavivienda_map)

tienetitulo_map = {
    1.0: 'Si',
    2.0: 'No',
    3.0: 'En tramite de titulacion',
    9.0: 'Missing Value'
}
df_final['TIENETITULO'] = df_final['TIENETITULO'].map(tienetitulo_map)

provenienciadeagua_map = {
    1.0: 'Red publica, dentro de la vivienda',
    2.0: 'Red publica, fuera de la vivienda pero dentro del edificio',
    3.0: 'Pilon o pileta de uso publico',
    4.0: 'Camion-cisterna',
    5.0: 'Pozo',
    6.0: 'Manantial o puquio',
    7.0: 'Otra',
    8.0: 'Rio, acequia, lago, laguna'
}
df_final['PROVENIENCIADEAGUA'] = df_final['PROVENIENCIADEAGUA'].map(provenienciadeagua_map)

internet_map = {
    1.0: 'Si',
    2.0: 'No'
}
df_final['INTERNET'] = df_final['INTERNET'].map(internet_map)

# Reordenar columnas
columnas_finales = ['ANHO', 'DEPARTAMENTO', 'TIPOVIVIENDA', 'PERTENENCIAVIVIENDA', 'TIENETITULO', 'PROVENIENCIADEAGUA', 'INTERNET']
df_final = df_final[columnas_finales]

# Verificar el DataFrame final
print(df_final.head())



   ANHO DEPARTAMENTO                                       TIPOVIVIENDA  \
0  2023     AMAZONAS  Vivienda en casa de vecindad (Callejon, solar ...   
1  2023     AMAZONAS                                 Casa independiente   
2  2023     AMAZONAS                                 Casa independiente   
3  2023     AMAZONAS  Vivienda en casa de vecindad (Callejon, solar ...   
4  2023     AMAZONAS  Vivienda en casa de vecindad (Callejon, solar ...   

         PERTENENCIAVIVIENDA TIENETITULO  \
0                  Alquilada          No   
1  Propia, totalmente pagada          Si   
2                  Alquilada          No   
3  Propia, totalmente pagada          Si   
4  Propia, totalmente pagada          Si   

                                  PROVENIENCIADEAGUA INTERNET  
0  Red publica, fuera de la vivienda pero dentro ...       Si  
1                 Red publica, dentro de la vivienda       Si  
2                 Red publica, dentro de la vivienda       Si  
3  Red publica, fuera de la 

In [ ]:
# Guardar el nuevo DataFrame en un archivo CSV
df_final.to_csv('df_vivienda.csv', index=False)

files.download('df_vivienda.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>